<a href="https://colab.research.google.com/github/JoseMauricioBazagoitiaAyllon/Inteligencia-Artificial-II/blob/main/BarlowTwins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Self-Supervised Learning*

Para tener claro este concepto se necesita poner claro los tipos de aprendizaje que son dos el aprendisaje supervisado y el aprendisaje no supervisado.

![](https://miro.medium.com/max/4416/1*bvMhd_xpVxfJYoKXYp5hug.png)

Dependiendo de los datos disponibles y la tarea que queramos abordar, podemos elegir entre distintos tipos de aprendizaje. Estos son: aprendizaje supervisado, aprendizaje no supervisado, aprendizaje semi-supervisado y aprendizaje por refuerzo.

* Aprendizaje supervisado 
El aprendizaje supervisado necesita conjuntos de datos etiquetados, es decir, le decimos al modelo qué es lo que queremos que aprenda, y basandonos en los resultados conseguimos podemos modificar los parametros para mejorar el modelo..
Dependiendo del tipo de etiqueta, dentro del aprendizaje supervisado existen dos tipos de modelos: 
-Modelos de clasificacion
-Modelos de regrecion
* Aprendizaje no supervisado 
El aprendizaje no supervisado trabaja con datos que no han sido etiquetados. No tenemos una etiqueta que predecir. Estos algoritmos se usan principalmente en tareas donde es necesario analizar los datos para extraer nuevo conocimiento o agrupar entidades por afinidad.
* Aprendizaje semi-supervisado 
Este tipo de aprendizaje tiene un poco de los dos anteriores. Usando este enfoque, se comienza etiquetando manualmente algunos datos. Una vez tenemos una pequeña porción de datos etiquetados, entrenamos uno o varios algoritmos de aprendizaje supervisado sobre esa pequeña parte de datos etiquetados y utilizamos los modelos resultantes del entrenamiento para etiquetar el resto de datos.
* Aprendizaje por refuerzo 
el aprendizaje por refuerzo es un método de aprendizaje automático que se basa en recompensar los comportamientos deseados y penalizar los no deseados.
Para profundizar los contextos podria vistar el proximo enlace https://datos.gob.es/es/blog/como-aprenden-las-maquinas-machine-learning-y-sus-diferentes-tipos



# *Barlow Twins*

![](https://user-images.githubusercontent.com/14848164/120419539-b0fab900-c330-11eb-8536-126ce6ce7b85.png)
Pra explicar este modelo esta imagen sera de mucha ayuda, 
en un principio se cuenta un dataset con imagenes sin etiquetaslo cul lo vuelve un modelo de aprendizaje no supervisado, como no se cuenta con etiquetas para medir la perdida este modelo obta por realizar un copia de de los datos, pero esta cpia no sera identica sino constara de de transformaciones laws cuales nos permitira contar con dos caminos que compara.
Esta imagenes que ya cuentan con la transformacion la yamaremos Y^A y Y^B.
cada una de estas imagenes serviran como entrada a para los distintos entrenamiento esto nos lanzara valores para cada dataSet que los de nominaremos embeddings estos enbeddings seran compparadas de nuevo para conseguir lo mas parecido a una matris identidad mediante una funcion denominada Empirical cross-correlation.
danto los resultados finales.
Para un mayor entendimiento contaremos con 2 ejemplos
https://github.com/facebookresearch/barlowtwins 
Ejemplo Git Hub
https://colab.research.google.com/github/juansensio/blog/blob/master/066_ssl/ssl.ipynb
Ejemplo de Sensio

y por ultimo una edicion del cuadernillo de sensio comentado
